In [3]:
import time
import networkx as nx
from shapely.geometry import LineString
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Start timing
start_time = time.time()

# Define the golden ratio
phi = (1 + np.sqrt(5)) / 2

# Define the vertices of the outer dodecahedron
vertices_outer = np.array([
    [1, 1, 1], [-1, 1, 1], [1, -1, 1], [-1, -1, 1],
    [1, 1, -1], [-1, 1, -1], [1, -1, -1], [-1, -1, -1],
    [0, 1/phi, phi], [0, -1/phi, phi], [0, 1/phi, -phi], [0, -1/phi, -phi],
    [1/phi, phi, 0], [-1/phi, phi, 0], [1/phi, -phi, 0], [-1/phi, -phi, 0],
    [phi, 0, 1/phi], [-phi, 0, 1/phi], [phi, 0, -1/phi], [-phi, 0, -1/phi]
])

# Calculate the center of the outer dodecahedron
center = np.mean(vertices_outer, axis=0)

# Create the inner dodecahedron by scaling and translating the outer one
scale_factor = 0.8
vertices_inner = (vertices_outer - center) * scale_factor + center

# Define the edges
edges = [
    (0, 16), (0, 8), (0, 12), (16, 2), (16, 18), (8, 1), (8, 9), (12, 4), (12, 13),
    (2, 14), (2, 9), (18, 4), (18, 6), (1, 17), (1, 13), (4, 10), (13, 5),
    (14, 6), (14, 15), (9, 3), (17, 3), (17, 19), (5, 10), (5, 19),
    (6, 11), (15, 3), (15, 7), (10, 11), (19, 7), (3, 7), (11, 7)
]

# Define source and end points
source = (-5, -1, 1)
end = (5, 1, -1)

# Initialize graph
G = nx.Graph()

# Add dodecahedron vertices as nodes
for vertex in np.vstack([vertices_outer, vertices_inner]):
    G.add_node(tuple(vertex))

# Add source and end points as nodes
G.add_node(source)
G.add_node(end)

# Calculate the center and radius of the inner dodecahedron
inner_center = np.mean(vertices_inner, axis=0)
inner_radius = np.max(np.linalg.norm(vertices_inner - inner_center, axis=1))

# Generate more rock-like points within the inner dodecahedron
num_rock_points = 200
rock_points = []
while len(rock_points) < num_rock_points:
    point = np.random.normal(loc=inner_center, scale=inner_radius/3, size=3)
    if np.linalg.norm(point - inner_center) <= inner_radius:
        rock_points.append(point)
rock_points = np.array(rock_points)

# Add rock points as nodes
for point in rock_points:
    G.add_node(tuple(point))

# Calculate points 0.1 units inside the small dodecahedron
distance_inside = 0.1

points_inside_inner_nodes = []
for vertex in vertices_inner:
    direction_to_center = inner_center - vertex
    direction_normalized = direction_to_center / np.linalg.norm(direction_to_center)
    new_point = vertex + distance_inside * direction_normalized
    points_inside_inner_nodes.append(new_point)

# Convert to a NumPy array for easier manipulation
points_inside_inner_nodes = np.array(points_inside_inner_nodes)

# Add these new points as nodes in the graph
for point in points_inside_inner_nodes:
    G.add_node(tuple(point))

# Function to add edges connecting the dodecahedron vertices
def add_edges_dodecahedron(graph, vertices, edges):
    for edge in edges:
        u, v = tuple(vertices[edge[0]]), tuple(vertices[edge[1]])
        graph.add_edge(u, v, weight=LineString([u, v]).length)

# Add edges to the dodecahedron vertices
add_edges_dodecahedron(G, vertices_outer, edges)
add_edges_dodecahedron(G, vertices_inner, edges)

# Function to check if two line segments intersect
def segments_intersect(seg1, seg2):
    line1 = LineString(seg1)
    line2 = LineString(seg2)
    return line1.intersects(line2) and not line1.touches(line2)

# Function to add edges from a point to dodecahedron vertices without intersecting dodecahedron edges
def add_edges_without_intersection(graph, point, vertices, dodecahedron_edges):
    dodecahedron_edges_coords = [(tuple(vertices[edge[0]]), tuple(vertices[edge[1]])) for edge in dodecahedron_edges]
    for vertex in vertices:
        intersects = False
        new_edge = (point, tuple(vertex))
        for edge in dodecahedron_edges_coords:
            if segments_intersect(new_edge, edge):
                intersects = True
                print(f"Edge from {point} to {vertex} intersects with dodecahedron edge {edge}")
                break
        if not intersects:
            weight = LineString(new_edge).length
            graph.add_edge(point, tuple(vertex), weight=weight)
            print(f"Added edge from {point} to {vertex}")

# Add edges for source and end points
print("Adding edges from source:")
add_edges_without_intersection(G, source, vertices_outer, edges)
print("\nAdding edges from end:")
add_edges_without_intersection(G, end, vertices_outer, edges)

# Define heuristic function for A* (Euclidean distance)
def heuristic(u, v):
    return np.linalg.norm(np.array(u) - np.array(v))

# Find the shortest path using A* algorithm
try:
    path = nx.astar_path(G, source, end, heuristic=heuristic, weight='weight')
    print("\nShortest path found by A* algorithm:")
    print(path)
except nx.NetworkXNoPath:
    print("\nNo path found from source to end.")

# End timing
end_time = time.time()
execution_time = end_time - start_time

print(f"\nExecution time (without plotting): {execution_time} seconds.")

Adding edges from source:
Edge from (-5, -1, 1) to [1. 1. 1.] intersects with dodecahedron edge ((0.0, 0.6180339887498948, 1.618033988749895), (-1.0, 1.0, 1.0))
Added edge from (-5, -1, 1) to [-1.  1.  1.]
Added edge from (-5, -1, 1) to [ 1. -1.  1.]
Added edge from (-5, -1, 1) to [-1. -1.  1.]
Edge from (-5, -1, 1) to [ 1.  1. -1.] intersects with dodecahedron edge ((0.0, 0.6180339887498948, 1.618033988749895), (-1.0, 1.0, 1.0))
Added edge from (-5, -1, 1) to [-1.  1. -1.]
Added edge from (-5, -1, 1) to [ 1. -1. -1.]
Added edge from (-5, -1, 1) to [-1. -1. -1.]
Edge from (-5, -1, 1) to [0.         0.61803399 1.61803399] intersects with dodecahedron edge ((-1.0, 1.0, 1.0), (-1.618033988749895, 0.0, 0.6180339887498948))
Edge from (-5, -1, 1) to [ 0.         -0.61803399  1.61803399] intersects with dodecahedron edge ((-1.618033988749895, 0.0, 0.6180339887498948), (-1.0, -1.0, 1.0))
Edge from (-5, -1, 1) to [ 0.          0.61803399 -1.61803399] intersects with dodecahedron edge ((-1.0, 1.